Вариант 2: Дан текст, разделенный на абзацы. Необходимо построить обратный индекс этого текста, показывающий в каком абзаце и на какой позиции хранится слово. Пользователь вводит два слова. Необходимо определить, находятся ли эти два слова в соседних абзацах. Если слова находятся в одном абзаце, необходимо найти расстояние между ними.

In [ ]:
import re

In [ ]:
def cleantext(text):
    punctuation = re.compile(r'[.?!,@#$%^&*()\"№:;\-–—+<>/\\«»]')
    clean = re.sub(punctuation, '', text)
    return clean.lower()

In [ ]:
text = 'input.txt'
with open(text, encoding = 'utf-8') as f:
    f = f.read()
    f = cleantext(f)
    paragraphs = f.split('\n')
    # Убираем пустые строки из списка абзацев
    paragraphs = [i for i in paragraphs if i]
# Создаём словарь, в котором ключи — номера абзацев, а значения — списки из слов этого абзаца
splittext = {}
i = -1
for paragraph in paragraphs:
    i += 1
    splitparagraph = paragraph.split()
    splittext[i] = splitparagraph

In [ ]:
def addPostfix(postfix, curLevel):    
    # Если в префиксе осталось только два элемента, то это наш индекс
    if len(postfix) == 2:  
        # Добавим словарь {'индексы':[[индекс 1], [индекс 2], [индекс 3]...]} для хранения множественных вхождений
        index = [postfix[0], postfix[1]]
        if curLevel.get('индексы', None) == None:
            curLevel.update({'индексы': [index]})
        elif index not in curLevel['индексы']:
            curLevel['индексы'].append(index)
        return
    # Такого начала нет в словаре. Его надо добавить.
    if curLevel.get((postfix[0]), None) == None:
        curLevel[postfix[0]] = {}
    # Добавляем оставшуюся часть в соответствующий узел.
    return addPostfix(postfix[1:], curLevel[postfix[0]])

In [ ]:
def findPrefix(prefix, curLevel):
    # Префикс пустой — остались только индексы.
    if len(prefix) == 0:
        return curLevel['индексы']
    # Такого слова нет в дереве
    elif curLevel.get(prefix[0], None) == None:
        return -1
    # Поиск не закончен, переходим на следующий уровень, убрав очередной символ из начала.
    return findPrefix(prefix[1:], curLevel[prefix[0]])

In [ ]:
# Составляем дерево
# В него добавляем слова по букве, последнее значение — список индексов слова
trie = {}
for num, words in splittext.items():
    for word in words:
        occurences = [i for i, x in enumerate(words) if x == word]
        for j in occurences:
            wordind = list(word)+[num,j]
            addPostfix(wordind, trie)

In [ ]:
# Поиск по одному слову
word = input('Введите слово: ')
indices = findPrefix(list(word),trie)
if indices == -1:
    print('такого слова нет')
else:
    print('это слово встречается на позициях', indices)

In [ ]:
# Поиск по двум словам
first = input('Первое слово — ')
second = input('Второе слово — ')
indices1 = findPrefix(list(first),trie)
indices2 = findPrefix(list(second),trie)
closest, nextpar = None, None
if indices1 != -1 and indices2 != -1:
    for i1 in indices1:
        for i2 in indices2:
            if i1[0] == i2[0]:
                if closest == None:
                    closest = abs(i1[1]-i2[1])
                elif closest > abs(i1[1]-i2[1]):
                    closest = abs(i1[1]-i2[1])
            elif abs(i1[0]-i2[0]) == 1:
                nextpar = [i1[0],i2[0]]
    if closest != None:
        print('слова встречаются в одном абзаце, наименьшее расстояние —', closest)
    elif nextpar:
        print('слова встречаются в соседних абзацах', nextpar[0], 'и', nextpar[1])
    else:
        print('слова не встречаются ни в одном, ни в соседних абзацах')
else:
    print('одного или обоих этих слов нет в тексте')